In [10]:
sample="""939
7,13,x,x,59,x,31,19
"""
my_in = """1001287
13,x,x,x,x,x,x,37,x,x,x,x,x,461,x,x,x,x,x,x,x,x,x,x,x,x,x,17,x,x,x,x,19,x,x,x,x,x,x,x,x,x,29,x,739,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,x,x,x,x,x,x,23
"""

In [11]:
def parse_input(s):
    aux = s.split('\n')
    return (int(aux[0]),
            [int(x) for x in aux[1].split(',') if x != 'x'])
assert parse_input(sample) == (939, [7, 13, 59, 31, 19])

In [12]:
def pick_bus1(time_start, bus_list):
    min_time = 0
    min_bus = None
    for bus in bus_list:
        w = time_start % bus
        aux = bus - w
        print(bus, w, aux)
        if min_bus is None or min_time > aux:
            min_bus = bus
            min_time = aux
    return min_bus, min_time
assert pick_bus1(*list(parse_input(sample))) == (59, 5)

def problem1(s):
    aux = pick_bus1(*list(parse_input(s)))
    print(aux)
    return aux[0] * aux[1]
assert problem1(sample) == 295

problem1(my_in)

7 1 6
13 3 10
59 54 5
31 9 22
19 8 11
7 1 6
13 3 10
59 54 5
31 9 22
19 8 11
(59, 5)
13 1 12
37 30 7
461 456 5
17 4 13
19 6 13
29 4 25
739 681 58
41 26 15
23 5 18
(461, 5)


2305

In [13]:
def parse_input2(s):
    aux = s.split('\n')
    return [(i, int(x)) for i, x in enumerate(aux[1].split(',')) if x != 'x']
assert parse_input2(sample) == [(0, 7), (1, 13), (4, 59), (6, 31), (7, 19)]
parse_input2(my_in)

[(0, 13),
 (7, 37),
 (13, 461),
 (27, 17),
 (32, 19),
 (42, 29),
 (44, 739),
 (54, 41),
 (67, 23)]

In [39]:
import time, math
def find_x_brute_force(list_input, x=0):
    """Find x which (x + i) % n = 0 for i, n in list_input"""
    init_time = time.time()
    PRINT_ON = 100_000_000
    while True:
        if x % PRINT_ON == -1 % PRINT_ON:
            print(x, time.time() - init_time)
        for i, n in list_input:
            if (x + i) % n != 0:
                x += 1
                break
        else:
            for i, n in list_input:
                print(x, n, i, x%n)
            return x
def find_x(list_input):
    """Find x which x % n = 0 for i, n in list_input
    x + i_0 % n_0 = 0 --> x = - i_0 (mod n_0)
    x + i_1 % n_1 = 0 --> x = - i_1 (mod n_1)
    ...
    Teorema dels residus xinesos
    
    """
    a = []
    n = []
    for i, x in list_input:
        n.append(x)
        a.append(-i)
    print(n, a)
    return chinese_remainder(n, a)

def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1
from functools import reduce
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(get_mcm, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod

def get_mcm(a, b):
    return a * b // math.gcd(a, b)
    
    
find_x(parse_input2(sample))

[7, 13, 59, 31, 19] [0, -1, -4, -6, -7]


1068781

In [40]:
find_x(parse_input2(my_in))

[13, 37, 461, 17, 19, 29, 739, 41, 23] [0, -7, -13, -27, -32, -42, -44, -54, -67]


552612234243498

In [41]:
def get_meta_bus(bus1, bus2):
    a0, n0 = bus1
    a1, n1 = bus2
    if n1 < n0:
        print('invert, because n0 is bigger')
        n0, n1 = n1, n0
        a0, a1 = a1, a0
    assert n1 >= n0
    n = get_mcm(n0, n1)
    a = - a1
    while (a + a0) % n0 != 0:
        a += n1
        assert a < n
    print('check both zero', (a+a0)%n0, (a+a1)%n1, a, n, a < n, bus1, bus2)
    return a % n, n

def find_x_my(listin):
    result = listin[0]
    for bus in listin:
        result = get_meta_bus(bus, result)
    for bus in listin:
        print('Check bus', bus, (result[0] + bus[0]) % bus[1])
    return result

In [42]:
find_x_my(parse_input2(my_in))

check both zero 0 0 0 13 True (0, 13) (0, 13)
invert, because n0 is bigger
check both zero 0 0 104 481 True (7, 37) (0, 13)
check both zero 0 0 101868 221741 True (13, 461) (104, 481)
check both zero 0 0 2780765 3769597 True (27, 17) (101868, 221741)
check both zero 0 0 12297623 71622343 True (32, 19) (2780765, 3769597)
check both zero 0 0 990415179 2077047947 True (42, 29) (12297623, 71622343)
check both zero 0 0 566043674352 1534938432833 True (44, 739) (990415179, 2077047947)
check both zero 0 0 34737540280807 62932475746153 True (54, 41) (566043674352, 1534938432833)
check both zero 0 0 1035114547403794 1447446942161519 True (67, 23) (34737540280807, 62932475746153)
Check bus (0, 13) 0
Check bus (7, 37) 14
Check bus (13, 461) 0
Check bus (27, 17) 3
Check bus (32, 19) 0
Check bus (42, 29) 26
Check bus (44, 739) 0
Check bus (54, 41) 26
Check bus (67, 23) 0


(1035114547403794, 1447446942161519)